# Convolutional Deep Symantic Similarity Model (CDSSM)
##Using TensorFlow and MatchZoo Toolkit

Install MatchZoo Toolkit

In [1]:
!pip3 install matchzoo

     |████████████████████████████████| 91 kB 3.1 MB/s 
  Created wheel for matchzoo: filename=MatchZoo-2.2.0-py3-none-any.whl size=151779 sha256=e432e32a6e2f836177a5e1771e6354470b36aee407e792584c7855c7b942c2b8
  Stored in directory: /root/.cache/pip/wheels/46/35/e3/70d4fccf0cfa8a617a50d37b3da011fe34bd977f7e126e9878
Successfully built matchzoo


import TensorFlow and keras

In [2]:
import tensorflow
from tensorflow import keras
print(keras.__version__) # Check Version

2.8.0


In [3]:
!python --version # Check Python Version

Python 3.7.12


Import Matchzoo Toolkit

In [13]:
import matchzoo as mz
mz.__version__

'2.2.0'

Import nltk and Download missing package dependencies

In [14]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Set Task to Ranking

In [15]:
task = mz.tasks.Ranking()
print(task)

Ranking Task


In [16]:
#from matchzoo.datasets import wiki_qa
#from matchzoo.tasks.ranking import Ranking
#ranking_task = Ranking()


In [17]:
train_pack = mz.datasets.wiki_qa.load_data(stage='train', task='ranking')
#valid_pack = mz.datasets.wiki_qa.load_data(stage='dev', task='ranking')
test_pack = mz.datasets.wiki_qa.load_data(stage='test', task='ranking')

#train_pack = mz.datasets.toy.load_data(stage='train', task=task)
#test_pack = mz.datasets.toy.load_data(stage='test', task=task)
type(train_pack)

7102464/7094233 [==============================] - 0s 0us/step


matchzoo.data_pack.data_pack.DataPack

In [18]:
train_pack.left.head()

,text_left
id_left,
Q1,how are glacier caves formed?
Q2,How are the directions of the velocity and for...
Q5,how did apollo creed die
Q6,how long is the term for federal judges
Q7,how a beretta model 21 pistols magazines works


In [19]:
train_pack.right.head()

,text_right
id_right,
D1-0,A partly submerged glacier cave on Perito More...
D1-1,The ice facade is approximately 60 m high
D1-2,Ice formations in the Titlis glacier cave
D1-3,A glacier cave is a cave formed within the ice...
D1-4,"Glacier caves are often called ice caves , but..."


In [20]:
train_pack.relation.head()

,id_left,id_right,label
0,Q1,D1-0,0
1,Q1,D1-1,0
2,Q1,D1-2,0
3,Q1,D1-3,1
4,Q1,D1-4,0


In [21]:
train_pack.frame().head()

,id_left,text_left,id_right,text_right,label
0,Q1,how are glacier caves formed?,D1-0,A partly submerged glacier cave on Perito More...,0
1,Q1,how are glacier caves formed?,D1-1,The ice facade is approximately 60 m high,0
2,Q1,how are glacier caves formed?,D1-2,Ice formations in the Titlis glacier cave,0
3,Q1,how are glacier caves formed?,D1-3,A glacier cave is a cave formed within the ice...,1
4,Q1,how are glacier caves formed?,D1-4,"Glacier caves are often called ice caves , but...",0


In [22]:
import pandas

In [23]:
#preprocessor = mz.models.CDSSM.get_default_preprocessor()
#preprocessor = mz.preprocessors.CDSSMPreprocessor(fixed_length_left=10, fixed_length_right=10, with_word_hashing=False)
#train_pack_processed = preprocessor.fit_transform(train_pack, verbose=0)
#valid_pack_processed = preprocessor.transform(valid_pack)
#test_pack_processed = preprocessor.transform(test_pack, verbose=0)
preprocessor = mz.preprocessors.BasicPreprocessor()
preprocessor.fit(train_pack)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8668.59it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4694.06it/s]
Building Vocabulary from a datapack.: 100%|██████████| 473697/473697 [00:00<00:00, 2682287.31it/s]


In [24]:
preprocessor.context

{'embedding_input_dim': 17483,
 'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f908a3f8d10>,
 'input_shapes': [(30,), (30,)],
 'vocab_size': 17483,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f908829a810>}

In [25]:
train_processed = preprocessor.transform(train_pack)
test_processed = preprocessor.transform(test_pack)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8729.64it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4442.73it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 84222.24it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 633/633 [00:00<00:00, 8433.83it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 5961/5961 [00:01<00:00, 4812.74it/s]
Processing text_right with transform: 100%|██████████| 5961/5961 [00:00<00:00, 78366.63it/s]


In [26]:
train_processed.left.head()

,text_left,length_left
id_left,,
Q1,"[5218, 3348, 10540, 10852, 8900, 2, 0, 0, 0, 0...",6
Q2,"[5218, 3348, 7288, 9516, 5959, 7288, 10151, 56...",15
Q5,"[5218, 9485, 11166, 14816, 921, 0, 0, 0, 0, 0,...",5
Q6,"[5218, 66, 11133, 7288, 15863, 11198, 14134, 9...",8
Q7,"[5218, 15899, 14237, 7146, 10691, 8967, 3346, ...",8


In [27]:
vocab_unit = preprocessor.context['vocab_unit']
print('Orig Text:', train_processed.left.loc['Q1']['text_left'])
sequence = train_processed.left.loc['Q1']['text_left']
print('Transformed Indices:', sequence)
print('Transformed Indices Meaning:',
      '_'.join([vocab_unit.state['index_term'][i] for i in sequence]))

Orig Text: [5218, 3348, 10540, 10852, 8900, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Transformed Indices: [5218, 3348, 10540, 10852, 8900, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Transformed Indices Meaning: how_are_glacier_caves_formed__<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>


In [28]:
mz.models.list_available()

[matchzoo.models.naive.Naive,
 matchzoo.models.dssm.DSSM,
 matchzoo.models.cdssm.CDSSM,
 matchzoo.models.dense_baseline.DenseBaseline,
 matchzoo.models.arci.ArcI,
 matchzoo.models.arcii.ArcII,
 matchzoo.models.match_pyramid.MatchPyramid,
 matchzoo.models.knrm.KNRM,
 matchzoo.models.duet.DUET,
 matchzoo.models.drmmtks.DRMMTKS,
 matchzoo.models.drmm.DRMM,
 matchzoo.models.anmm.ANMM,
 matchzoo.models.mvlstm.MVLSTM,
 matchzoo.contrib.models.match_lstm.MatchLSTM,
 matchzoo.contrib.models.match_srnn.MatchSRNN,
 matchzoo.contrib.models.hbmp.HBMP,
 matchzoo.contrib.models.esim.ESIM,
 matchzoo.contrib.models.bimpm.BiMPM,
 matchzoo.contrib.models.diin.DIIN,
 matchzoo.models.conv_knrm.ConvKNRM]

In [29]:
model = mz.models.DenseBaseline()
#model.params['input_shapes'] = preprocessor.context['input_shapes']
#model.params['task'] = mz.tasks.Ranking()
#model.params['filters'] = 64
#model.params['kernel_size'] = 3
#model.params['strides'] = 1
#model.params['padding'] = 'same'
#model.params['conv_activation_func'] = 'tanh'
#model.params['w_initializer'] = 'glorot_normal'
#model.params['b_initializer'] = 'zeros'
#model.params['mlp_num_layers'] = 1
#model.params['mlp_num_units'] = 64
#model.params['mlp_num_fan_out'] = 64
#model.params['mlp_activation_func'] = 'tanh'
#model.params['dropout_rate'] = 0.8
#model.params['optimizer'] = 'adadelta'
#model.guess_and_fill_missing_params()


In [30]:
print(model.params)

model_class                   <class 'matchzoo.models.dense_baseline.DenseBaseline'>
input_shapes                  None
task                          None
optimizer                     adam
with_multi_layer_perceptron   True
mlp_num_units                 256
mlp_num_layers                3
mlp_num_fan_out               64
mlp_activation_func           relu


In [31]:
model.params.to_frame()[['Name', 'Description', 'Value']]

,Name,Description,Value
0,model_class,Model class. Used internally for save/load. Ch...,<class 'matchzoo.models.dense_baseline.DenseBa...
1,input_shapes,Dependent on the model and data. Should be set...,None
2,task,"Decides model output shape, loss, and metrics.",None
3,optimizer,None,adam
4,with_multi_layer_perceptron,A flag of whether a multiple layer perceptron ...,True
5,mlp_num_units,Number of units in first `mlp_num_layers` layers.,256
6,mlp_num_layers,Number of layers of the multiple layer percetron.,3
7,mlp_num_fan_out,Number of units of the layer that connects the...,64
8,mlp_activation_func,Activation function used in the multiple layer...,relu


In [32]:
model.params['task'] = task
model.params['mlp_num_units'] = 3
print(model.params)

model_class                   <class 'matchzoo.models.dense_baseline.DenseBaseline'>
input_shapes                  None
task                          Ranking Task
optimizer                     adam
with_multi_layer_perceptron   True
mlp_num_units                 3
mlp_num_layers                3
mlp_num_fan_out               64
mlp_activation_func           relu


In [33]:
print(preprocessor.context['input_shapes'])

[(30,), (30,)]


In [34]:
model.params.update(preprocessor.context)

In [ ]:
model.params.completed()

In [36]:
model.build()
model.compile()

In [37]:
model.backend.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text_left (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 text_right (InputLayer)        [(None, 30)]         0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 60)           0           ['text_left[0][0]',              
                                                                  'text_right[0][0]']             
                                                                                                  
 dense (Dense)                  (None, 3)            183         ['concatenate[0][0]']        

In [38]:
x, y = train_processed.unpack()
test_x, test_y = test_processed.unpack()

In [39]:
model.fit(x, y, batch_size=32, epochs=5)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['id_left', 'length_left', 'id_right', 'length_right'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


637/637 [==============================] - 2s 2ms/step - loss: 44983.6289
Epoch 2/5
637/637 [==============================] - 1s 2ms/step - loss: 0.0556
Epoch 3/5
637/637 [==============================] - 1s 2ms/step - loss: 0.0556
Epoch 4/5
637/637 [==============================] - 1s 2ms/step - loss: 0.0555
Epoch 5/5
637/637 [==============================] - 1s 2ms/step - loss: 0.0559


In [ ]:
#pred_x, pred_y = test_pack_processed[:].unpack()
#evaluate = mz.callbacks.EvaluateAllMetrics(model, x=x, y=y, batch_size=len(x))
#train_generator = mz.DataGenerator(
#    train_pack,
#    mode='pair',
#    num_dup=2,
#    num_neg=1,
#    batch_size=20
#)
#print('num batches:', len(train_generator))

In [40]:
data_generator = mz.DataGenerator(train_processed, batch_size=32)

In [41]:
model.fit_generator(data_generator, epochs=5, use_multiprocessing=True, workers=4)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/matchzoo/engine/base_model.py:276: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  verbose=verbose, **kwargs
/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['id_left', 'length_left', 'id_right', 'length_right'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


637/637 [==============================] - 4s 6ms/step - loss: 0.0558
Epoch 2/5
637/637 [==============================] - 4s 5ms/step - loss: 0.0560
Epoch 3/5
637/637 [==============================] - 4s 5ms/step - loss: 0.0560
Epoch 4/5
637/637 [==============================] - 4s 5ms/step - loss: 0.0591
Epoch 5/5
637/637 [==============================] - 5s 7ms/step - loss: 0.0576


In [42]:
model.evaluate(test_x, test_y)

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['id_left', 'length_left', 'id_right', 'length_right'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


{mean_average_precision(0.0): 0.1620382723689128}

In [51]:
# m = model.predict(test_x)
model.predict(test_x)

array([[-0.02525885],
       [-0.03764899],
       [-0.01915533],
       ...,
       [-0.01134284],
       [-0.04674323],
       [-0.01939948]], dtype=float32)

In [57]:
len(test_x)

6

In [52]:
import torch
prob = torch.softmax(0.022,dim=-3)

TypeError: ignored

In [ ]:
for model_class in mz.models.list_available():
    print(model_class)
    model, preprocessor, data_generator_builder, embedding_matrix = mz.auto.prepare(
        task=task,
        model_class=model_class,
        data_pack=train_pack,
    )
    train_processed = preprocessor.transform(train_pack, verbose=0)
    test_processed = preprocessor.transform(test_pack, verbose=0)
    train_gen = data_generator_builder.build(train_processed)
    test_gen = data_generator_builder.build(test_processed)
    model.fit_generator(train_gen, epochs=1)
    model.evaluate_generator(test_gen)
    print()

<class 'matchzoo.models.naive.Naive'>


/usr/local/lib/python3.7/dist-packages/matchzoo/engine/base_model.py:276: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  verbose=verbose, **kwargs
/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['id_left', 'length_left', 'id_right', 'length_right'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


32/32 [==============================] - 1s 15ms/step - loss: 33603576.0000

<class 'matchzoo.models.dssm.DSSM'>


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['id_left', 'id_right'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


32/32 [==============================] - 13s 389ms/step - loss: 0.2458

<class 'matchzoo.models.cdssm.CDSSM'>
32/32 [==============================] - 331s 10s/step - loss: 0.2263


KeyboardInterrupt: ignored

In [ ]:
#import tensorflow as tf
#train_generator=tf.convert_to_tensor(train_generator, dtype=None, dtype_hint=None, name=None)
#history = model.fit_generator(train_generator, epochs=20, callbacks=[evaluate], workers=1, use_multiprocessing=False)